#Análise Exploratória de Dados de Logística


##1. Contexto


O código em questão foi desenvolvido para a empresa [Loggi](https://www.loggi.com), que é uma plataforma de logística sob demanda que conecta motoristas e empresas a uma rede de entregas de alta qualidade, buscando soluções para tornar a entrega mais eficiente e rápida.

O objetivo deste código é realizar a limpeza e pré-processamento de dados para análise dos registros de entregas realizadas pela Loggi. O conjunto de dados contém informações como nome da região, origem, capacidade do veículo, latitude e longitude, tamanho do delivery, latitude e longitude do delivery.

Através deste código, foram realizadas algumas transformações, tais como a renomeação de colunas, a exploração do campo de deliveries, normalização de campos para possibilitar a análise, seleção de determinadas colunas e a exclusão de outras. A análise exploratória dos dados permitiu obter informações relevantes sobre o perfil das entregas realizadas e a performance dos entregadores, auxiliando a Loggi em suas estratégias de melhoria do serviço prestado. 😛 

Este notebook está disponivel no [kaggle](https://www.kaggle.com/leonardopulpor/an-lise-explorat-ria-de-dados-de-log-stica)!

##2. Pacotes e bibliotecas

In [47]:
!pip install geopandas;
!pip install geopy;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

##3. Exploração de dados

In [75]:
# !wget baixa o arquivo do URL 
# parâmetro -q executa o comando em modo silencioso, sem imprimir informações no console
# <<EOF é uma sintaxe usada para criar um bloco de texto em Python.  (foi retirado, pois estava dando mto problema)
# parâmetro -O é usado para dar nome ao arquivo de saída. No caso, o arquivo será salvo com o nome "deliveries.json".

!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json

In [76]:
import json

with open('deliveries.json', mode='r', encoding='utf8') as file:
    data = json.load(file)

In [77]:
len(data)

199

In [104]:
df = pd.DataFrame(data)
df.head(2)

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."


In [99]:
df = data[0]
print(df.keys())

dict_keys(['name', 'region', 'origin', 'vehicle_capacity', 'deliveries'])


In [100]:
# iterando sobre os 10 elementos do objeto JSON

for i in range(10):
    delivery = data[i]

    print("Nome: ", delivery['name'])
    print("Região: ", delivery['region'])
    print("Origem: ", delivery['origin'])
    print("Capacidade de veículos: ", delivery['vehicle_capacity'])
    print("Delivery: ", delivery['deliveries'][i])
 
    print("--------------------------")

Nome:  cvrp-2-df-33
Região:  df-2
Origem:  {'lng': -48.05498915846707, 'lat': -15.83814451122274}
Capacidade de veículos:  180
Delivery:  {'id': '313483a19d2f8d65cd5024c8d215cfbd', 'point': {'lng': -48.11618888384239, 'lat': -15.848929154862294}, 'size': 9}
--------------------------
Nome:  cvrp-2-df-73
Região:  df-2
Origem:  {'lng': -48.05498915846707, 'lat': -15.83814451122274}
Capacidade de veículos:  180
Delivery:  {'id': 'd6dcb5d6c0cb6f1f904b59ac64e1f531', 'point': {'lng': -48.118629353903, 'lat': -15.852619700821812}, 'size': 7}
--------------------------
Nome:  cvrp-2-df-20
Região:  df-2
Origem:  {'lng': -48.05498915846707, 'lat': -15.83814451122274}
Capacidade de veículos:  180
Delivery:  {'id': 'b30cad06a6593e76aae9258c730d6d17', 'point': {'lng': -48.11813310985553, 'lat': -15.846928255884968}, 'size': 4}
--------------------------
Nome:  cvrp-1-df-71
Região:  df-1
Origem:  {'lng': -47.89366206897872, 'lat': -15.80511751066334}
Capacidade de veículos:  180
Delivery:  {'id': 'd

In [105]:
#coluna origin tem dados aninhados

# método 'pd.json_normalize()' do pandas transforma o dic df["origin"] em um DataFrame pandas.
# é usado para normalizar/achatar (flatten) dados JSON em um DataFrame Pandas. 
# converte dados JSON em um DataFrame de formato tabular, em que cada coluna representa um campo ou chave do JSON e cada linha representa um objeto JSON.

hub_origin_df = pd.json_normalize(df["origin"])
hub_origin_df.head()

,lng,lat
0,-48.054989,-15.838145
1,-48.054989,-15.838145
2,-48.054989,-15.838145
3,-47.893662,-15.805118
4,-48.054989,-15.838145


In [118]:
# mescla 'df' e 'hub_origin_df' (2 df) usando o índice das duas tabelas 'left_index=True', 'right_index=True'
# e o método de mesclagem inner join (how='inner')
# cria um novo DataFrame que contém ambas as tabelas, com base no índice em comum. 


df = pd.merge(left=df, right=hub_origin_df, how='inner', left_index=True, right_index=True)
df.head()

,name,region,origin,vehicle_capacity,deliveries,lng,lat
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145


In [116]:
#removendo origin, passou a ser irrelevante
#dois colchetes cria-se um novo df, se fosse apenas um ele simplesmente 'acessaria' a coluna

df = df.drop("origin", axis=1)
df = df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]
df.head()


,name,region,lng,lat,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,-48.054989,-15.838145,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,-48.054989,-15.838145,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,-47.893662,-15.805118,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,-48.054989,-15.838145,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


In [119]:
df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"},inplace=True)
df.head()

,name,region,origin,vehicle_capacity,deliveries,hub_lng,hub_lat
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145


arrumando a coluna delivieres que tem dados aninhados (nested)

In [120]:
deliveries_exploded_df = df[["deliveries"]].explode("deliveries")
deliveries_exploded_df.head()


,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."


In [122]:
# dividindo a coluna "deliveries" do df "deliveries_exploded_df" em 3 colunas: "delivery_size", "delivery_lng" e "delivery_lat".
# método "apply" do pandas c função lambda, q estão acessando as informações do dic em cada dado da coluna "deliveries" e retornando os valores das chaves "size", "point.lng" e "point.lat".
# concatenação horizontal (eixo 1) utilizando o método "pd.concat"


deliveries_normalized_df = pd.concat([
 pd.DataFrame(deliveries_exploded_df["deliveries"].apply(
 lambda record: record["size"])
).rename(columns={"deliveries": "delivery_size"}),
 pd.DataFrame(deliveries_exploded_df["deliveries"].apply(
 lambda record: record["point"]["lng"])
).rename(columns={"deliveries": "delivery_lng"}),
 pd.DataFrame(deliveries_exploded_df["deliveries"].apply(
 lambda record: record["point"]["lat"])
).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)
deliveries_normalized_df.head()

,delivery_size,delivery_lng,delivery_lat
0,9,-48.116189,-15.848929
0,2,-48.118195,-15.850772
0,1,-48.112483,-15.847871
0,2,-48.118023,-15.846471
0,7,-48.114898,-15.858055


In [123]:
len(deliveries_exploded_df)

636149

In [125]:
len(df)

199

In [126]:
# removendo a coluna "deliveries" do dataframe 'df' e mesclando com o dataframe 'deliveries_normalized_df' utilizando o método pd.merge().
# parâmetro how='right' mantem todas as linhas do dataframe 'deliveries_normalized_df', mesmo que elas não tenham correspondência com o dataframe 'df'.
# parâmetro left_index=True e right_index=True dizem que a mesclagem será feita com base no índice de *AMBOS* os dataframes.
# reseta o índice do dataframe novo com df.reset_index(inplace=True, drop=True).

# O resultado é um dataframe q tem todas as informações de 'df' e as informações normalizadas de 'deliveries_exploded_df' (tamanho da entrega, a longitude e a latitude)


df = df.drop("deliveries", axis=1)
df = pd.merge(left=df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
df.reset_index(inplace=True, drop=True)

df.head()

,name,region,origin,vehicle_capacity,hub_lng,hub_lat,delivery_size,delivery_lng,delivery_lat
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,9,-48.116189,-15.848929
1,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,2,-48.118195,-15.850772
2,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,1,-48.112483,-15.847871
3,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,2,-48.118023,-15.846471
4,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,7,-48.114898,-15.858055


In [127]:
len(df)

636149

In [129]:
#linhas, colunas
df.shape

(636149, 9)

In [130]:
df.columns

Index(['name', 'region', 'origin', 'vehicle_capacity', 'hub_lng', 'hub_lat',
       'delivery_size', 'delivery_lng', 'delivery_lat'],
      dtype='object')

In [137]:
df = df.rename(columns={
    
                        "name": "nome",
                        "region": "região",
                        "origin": "origem", 
                        "vehicle_capacity": "capacidade do veiculo",
                        "hub_lng": "longitude",
                        "hub_lat": "latitude",                
                        "delivery_size": "tamanho do delivery",
                        "delivery_lng": "latitude do delivery",
                        "delivery_lat": "longitude do delivery",
                        
                        })

In [138]:
df

,nome,região,origem,capacidade_do_veiculo,longitude,latitude,tamanho do delivery,latitude do delivery,longitude do delivery
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,9,-48.116189,-15.848929
1,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,2,-48.118195,-15.850772
2,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,1,-48.112483,-15.847871
3,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,2,-48.118023,-15.846471
4,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,7,-48.114898,-15.858055
...,...,...,...,...,...,...,...,...,...
636144,cvrp-2-df-62,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,8,-48.064269,-15.997694
636145,cvrp-2-df-62,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,4,-48.065176,-16.003597
636146,cvrp-2-df-62,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,9,-48.065841,-16.003808
636147,cvrp-2-df-62,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,-48.054989,-15.838145,1,-48.062327,-16.001568


In [141]:
#index retorna o índice do DataFrame, ou seja, uma estrutura que permite identificar cada linha da tabela de dados de forma única (tipo uma coordenada)

df.index

RangeIndex(start=0, stop=636149, step=1)

In [142]:
#info é um método que retorna informações sobre o DataFrame,  número de linhas, número de colunas, nome das colunas, tipo de dado de cada coluna e quantidade de valores não nulos. É uma forma rápida de ter uma visão geral dos dados contidos no DataFrame.

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636149 entries, 0 to 636148
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   nome                   636149 non-null  object 
 1   região                 636149 non-null  object 
 2   origem                 636149 non-null  object 
 3   capacidade_do_veiculo  636149 non-null  int64  
 4   longitude              636149 non-null  float64
 5   latitude               636149 non-null  float64
 6   tamanho do delivery    636149 non-null  int64  
 7   latitude do delivery   636149 non-null  float64
 8   longitude do delivery  636149 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 43.7+ MB


In [144]:
#mostra o tipo de dado da coluna
df.dtypes


nome                      object
região                    object
origem                    object
capacidade_do_veiculo      int64
longitude                float64
latitude                 float64
tamanho do delivery        int64
latitude do delivery     float64
longitude do delivery    float64
dtype: object

In [145]:
# resumo estatístico das colunas do dataframe que são 'object' (strings). 
# método describe() calcula estatísticas das colunas, como contagem, número de valores únicos, valor mais frequente e sua frequência, etc.
# método transpose() é usado para transpor as linhas e colunas do resultado do describe(), para q cada coluna se torne uma linha no resultado.

df.select_dtypes("object").describe().transpose()

#obs: coluna unique, são valores unicos q aparecem só uma vez 

,count,unique,top,freq
nome,636149,199,cvrp-1-df-87,5636
região,636149,3,df-1,304708
origem,636149,3,"{'lng': -47.89366206897872, 'lat': -15.8051175...",304708


In [148]:
# drop remove as colunas (axis =1 é coluna e axis=0 é linha)
# estatísticas dos inteiros ( média, desvio padrão, mínimo, máximo, quartis, etc.)

df.drop(
    ["nome", "região"], axis=1
        ).select_dtypes('int64').describe().transpose()

,count,mean,std,min,25%,50%,75%,max
capacidade_do_veiculo,636149.0,180.000000,0.000000,180.0,180.0,180.0,180.0,180.0
tamanho do delivery,636149.0,5.512111,2.874557,1.0,3.0,6.0,8.0,10.0


In [149]:
df.isna().any()

nome                     False
região                   False
origem                   False
capacidade_do_veiculo    False
longitude                False
latitude                 False
tamanho do delivery      False
latitude do delivery     False
longitude do delivery    False
dtype: bool

O resultado acima diz que não há valores faltantes em nenhuma colunas do dataframe df. 

A expressão 'df.isna()' retorna uma tabela booleana com *'True'* para as posições onde **há valores faltantes** e *'False'* onde **não há**. 

O método 'any()' aplicado na tabela retorna 'True' se houver algum valor 'True' em uma das colunas e 'False' caso contrário.
Como o resultado de df.isna().any() é 'False' para todas as colunas, conclui-se que **não há valores faltantes em df.** 😉 